# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
# choose a name for experiment
experiment_name = 'exp-automl-housing'
project_folder = '.'

experiment = Experiment(ws, experiment_name)
experiment

In [ ]:
#files and directories
cc=os.getcwd
os.listdir(os.curdir)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "housing"
description_text = "California Housing Dataset for Udacity project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = ''
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


data = dataset.to_pandas_dataframe()
data.describe()

In [ ]:
def prepare_data(data):    
    encoded_column=pd.get_dummies(data['ocean_proximity'], prefix='ocp')
    data=data.join(encoded_column)
    data=data.drop("ocean_proximity", axis=1)

    target="median_house_value"
    y=data[target]
    X=data.drop(target, axis=1)
    
    return x,y

x,y = prepare_data(data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train['y']=y_train
x_test['y']=y_test

In [ ]:
columns=x_train.columns
x_train.to_csv(path_or_buf='trainset.csv', columns=columns, header=True, index=False) 
x_test.to_csv(path_or_buf='testset.csv', columns=columns, header=True, index=False)

In [ ]:
datastore=ws.get_default_datastore() 
datastore.upload_files(['trainset.csv'])

In [ ]:
train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])

datastore.upload_files(['testset.csv'])

In [ ]:
test=TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
# TODO: Put your automl config here

automl_setting={
    "featurization": "auto",
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
    "compute_target": compute_target
}

task="regression" 
automl_config = AutoMLConfig( 
    task=task, 
    primary_metric='mae', 
    training_data=train, 
    validation_data = test, 
    label_column_name='median_house_value', 
    **automl_setting
)

In [ ]:
# TODO: Submit your experiment
automl_run = exp.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()

In [ ]:
automl_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
import joblib  
joblib.dump(fitted_automl_model, "automl_model_housing.joblib")

In [ ]:
os.listdir(os.curdir)

In [ ]:
#automl_run_metrics = automl_run.get_metrics()
automl_run_metrics

In [ ]:
print('Bets Run ID', automl_run.id)
print('\n MAE', automl_run_metrics['mae'])

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# https://www.bluegranite.com/blog/train-and-deploy-machine-learning-models-using-the-azureml-service
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.ipynb

In [ ]:
# register the model
automl_model = run.register_model(model_name='automl_model_housing',
                           model_path='automl_model_housing.joblib')
print(model.name, model.id, model.version, sep='\t')

In [ ]:
from azureml.core.model import Model
model=Model(ws, 'automl_model_housing')

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "automl_model_housing.joblib")

os.stat(file_path)

Deploy in an Azure Container Instance

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service